# ONNX Fake News Detection — API Usage Notebook

This notebook demonstrates how to *use* the API layer defined in  
`ONNX_Fake_News_Detection_utils.py`.

 **No training is done here.**  
We assume a trained model already exists in `models/`.

We will show:
- Loading tokenizer
- Running ONNX prediction
- Using utility functions

This is a **contract demonstration**, not an end-to-end example.


## Utility Functions: Inputs and Outputs

This project exposes a small set of utility functions that form the internal
programming interface. Each function has a clear responsibility and a stable
input/output contract.

### Data Utilities
- **`load_fake_real_news()`**
  - **Input:** None (uses fixed dataset paths)
  - **Output:** `pandas.DataFrame`
    - `text` (str): Combined title and article text
    - `label` (int): `0 = fake`, `1 = real`

- **`compute_classification_metrics(y_true, y_pred)`**
  - **Input:** 
    - `y_true` (array-like): Ground truth labels
    - `y_pred` (array-like): Predicted labels
  - **Output:** `dict` with accuracy, precision, recall, F1, and confusion matrix

---

### LSTM Training & Conversion
- **`train_lstm_model(num_samples=None)`**
  - **Input:** 
    - `num_samples` (int, optional): Limit training data size
  - **Output:** `dict`
    - `history`: Training history
    - `metrics`: Evaluation metrics on test split

- **`convert_lstm_to_onnx()`**
  - **Input:** None (uses saved Keras model)
  - **Output:** `str` — Path to exported ONNX model

---

### Inference
- **`predict_lstm_onnx(texts)`**
  - **Input:** 
    - `texts` (List[str]): Raw news articles
  - **Output:** `List[dict]`
    - `label` (int): Prediction (`0 = fake`, `1 = real`)
    - `score` (float): Confidence score

---

### API Layer
- **`create_fastapi_app(model_type="lstm")`**
  - **Input:** 
    - `model_type` (str): Model backend (`"lstm"`)
  - **Output:** `FastAPI` application instance

The API layer is intentionally narrow: consumers interact only with inference
while training and conversion remain internal concerns.


In [9]:
from ONNX_Fake_News_Detection_utils import (
    FakeNewsRequest,
    create_fastapi_app,
)

## API Contract

The API exposes a single inference operation.

### Input
- `text` (string): Raw news article text

### Output
- `label` (int):
  - `0` → Fake
  - `1` → Real
- `score` (float): Model confidence

This contract is intentionally minimal to keep the API stable.


## Creating the API Instance

The API is created through a factory function.
Internally, this loads:
- The ONNX Runtime session
- The tokenizer used during training

The caller does not manage model state.


In [11]:
app = create_fastapi_app(model_type="lstm")
app


## Calling the API Internally

To keep this notebook self-contained, we invoke the FastAPI endpoint
directly instead of starting a web server.

This simulates how a client would interact with the API.


In [12]:
import httpx
from httpx import ASGITransport
from ONNX_Fake_News_Detection_utils import FakeNewsRequest

# Create the data object
req_data = {"text": "Government announces new economic reforms to stabilize markets."}

# Use AsyncClient with ASGITransport
# This explicitly tells httpx to communicate directly with your FastAPI 'app'
transport = ASGITransport(app=app)

async with httpx.AsyncClient(transport=transport, base_url="http://test") as client:
    response = await client.post("/predict", json=req_data)

print(response.json())

{'label': 0, 'score': 0.0006501972675323486}


## Design Notes

- The API exposes **only inference**
- All preprocessing is handled internally
- ONNX Runtime ensures fast, portable execution
- The interface is framework-agnostic

This makes the API suitable for:
- Web services
- Batch inference pipelines
- Integration into larger systems


## Out of Scope

This notebook intentionally excludes:
- Model training
- Dataset access
- Evaluation metrics
- Performance benchmarking

Those are demonstrated in the `ONNX_Fake_News_Detection.example.ipynb` notebook.
